# **RandomForest baseline code**

## 이 프로젝트는 DNA sequences과 ΔG(cf) - ΔG(lin) 열을 전처리하여 머신러닝 모델로 학습하는 코드입니다.
- 입력: DNA sequences
- 출력: ΔG(cf) - ΔG(lin)

### 먼저, Google 시트로 되어 있는 데이터셋 파일을 드라이브 폴더에 CSV 파일로 저장합니다.

In [ ]:
# 내 드라이브에 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Data Preprocessing**

### ΔG(cf) - ΔG(lin) 열을 등급으로 분류

1. ΔG(cf) - ΔG(lin) 열을 절대 값으로 계산하기

In [ ]:
import pandas as pd

# csv 파일 불러오기
csv_path = "/content/drive/MyDrive/mini_project/Karl/data/dataset-palindrome.csv" # your dataset location path

df = pd.read_csv(csv_path)

# ΔG(cf) - ΔG(lin) 열을 전처리하여 차이를 절대값으로 계산
def compute_abs_difference(val):
    try:
        parts = str(val).split("-")
        if len(parts) == 2:
            num1 = float(parts[0])
            num2 = float(parts[1])
            return abs(num1 - num2)
    except:
        return None

# 절대값 차이를 새로운 열로 추가
df["ΔG_abs_diff"] = df["ΔG(cf) - ΔG(lin)"].apply(compute_abs_difference)

# 결과 확인
print(df[["ΔG(cf) - ΔG(lin)", "ΔG_abs_diff"]].head())

  ΔG(cf) - ΔG(lin)  ΔG_abs_diff
0       2.77-28.33        25.56
1       2.06-25.74        23.68
2       2.45-30.51        28.06
3       1.31-26.71        25.40
4       1.59-23.09        21.50


In [ ]:
# 데이터 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Traits                  309 non-null    object 
 1   Location                310 non-null    object 
 2   Sequences               308 non-null    object 
 3   Size                    308 non-null    float64
 4   Found # of palindromes  308 non-null    float64
 5   ΔG(cf) - ΔG(lin)        308 non-null    object 
 6   Length of Palindrome    308 non-null    object 
 7   Amounts by length       307 non-null    object 
 8   Length of Spacers       308 non-null    object 
 9   Amounts by spacer       307 non-null    object 
 10  Numebr of Mismatch      308 non-null    object 
 11  Amounts by mismatches   307 non-null    object 
 12  ΔG_abs_diff             308 non-null    float64
dtypes: float64(3), object(10)
memory usage: 31.6+ KB


### csv 파일의 실제 행 번호를 보려면 인덱스에 2를 더하세요(예: 115 + 2 = 117 행 확인)

In [ ]:
# ΔG_abs_diff 열에서 결측행 찾기
failed_rows = df[df["ΔG_abs_diff"].isna()]

# 결측행의 ΔG(cf) - ΔG(lin) 값 보기
print("Rows where ΔG_abs_diff calculation failed:")
print(failed_rows[["ΔG(cf) - ΔG(lin)"]])

❗ Rows where ΔG_abs_diff calculation failed:
    ΔG(cf) - ΔG(lin)
192              NaN
289              NaN


### 파일에서 117번과 304번 행을 수정하고 다음으로 진행

2. ΔG_abs_diff 열의 값들을 그룹화

In [ ]:
# 그룹화 함수
def assign_group(val):
    if pd.isna(val):
        return "Unknown"
    elif 0 <= val < 7:
        return "1"
    elif 7 <= val < 14:
        return "2"
    elif 14 <= val < 21:
        return "3"
    elif 21 <= val < 28:
        return "4"
    elif 28 <= val < 35:
        return "5"
    else:
        return "Other"

# 열 추가
df["ΔG_group"] = df["ΔG_abs_diff"].apply(assign_group)

# 결과 확인
print(df[["ΔG_abs_diff", "ΔG_group"]].head(10))

   ΔG_abs_diff ΔG_group
0        25.56        4
1        23.68        4
2        28.06        5
3        25.40        4
4        21.50        4
5        24.51        4
6        20.09        3
7        23.62        4
8        22.02        4
9        20.43        3


3. ΔG_group 열의 값을 모델의 출력으로 사용하기 위해 수치형으로 변경

In [ ]:
# 값을 정수로 변환하기
df["ΔG_group"] = pd.to_numeric(df["ΔG_group"], errors="coerce")

4. 파일 저장

In [ ]:
# 파일 저장하기
save_path = "/content/drive/MyDrive/mini_project/Karl/data/palindrome_with_group.csv" # 파일 저장을 위한 경로
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print("File saved to:", save_path)

✅ File saved to: /content/drive/MyDrive/mini_project/Karl/data/palindrome_with_group.csv


In [ ]:
# 데이터 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Traits                  309 non-null    object 
 1   Location                310 non-null    object 
 2   Sequences               308 non-null    object 
 3   Size                    308 non-null    float64
 4   Found # of palindromes  308 non-null    float64
 5   ΔG(cf) - ΔG(lin)        308 non-null    object 
 6   Length of Palindrome    308 non-null    object 
 7   Amounts by length       307 non-null    object 
 8   Length of Spacers       308 non-null    object 
 9   Amounts by spacer       307 non-null    object 
 10  Numebr of Mismatch      308 non-null    object 
 11  Amounts by mismatches   307 non-null    object 
 12  ΔG_abs_diff             308 non-null    float64
 13  ΔG_group                307 non-null    float64
dtypes: float64(4), object(10)
memory usage: 34

### 위의 df.info() 결과를 통해 ΔG_group 열에 null이 아닌 값이 307개가 있다는 것을 확인할 수 있으며, 값 하나가 누락되었다.
- ΔG_abs_diff = 308, ΔG_group = 307
- 실수 또는 오류에 의해 잘못된 값이 들어갔을 것으로 예상되어 편의상 1개의 데이터는 제거, 제거 코드는 아래에서 진행

# **DNA Sequences encoding**

1. Seqeunces 열 확인

In [ ]:
# csv 파일 불러오기
df = pd.read_csv("/content/drive/MyDrive/mini_project/Karl/data/palindrome_with_group.csv")

# ΔG_abs_diff에 값이 있지만 ΔG_group이 NaN인 행 찾기
ungrouped = df[df["ΔG_abs_diff"].notna() & df["ΔG_group"].isna()]

# 출력
print("Rows that were not grouped:")
print(ungrouped[["ΔG(cf) - ΔG(lin)", "ΔG_abs_diff", "ΔG_group"]])

# ΔG_group 열에서 결측값이 있는 행 제거
df = df.dropna(subset=["ΔG_group"])

# 결측행 제거하고 문자열로 변환
sequences = df["Sequences"].dropna().astype(str)

# 모든 행의 서열을 하나로 합치고, 고유 문자 추출
all_chars = set("".join(sequences))

# 결과 출력
print("Number of unique characters:", len(all_chars))
print("List of unique characters:", sorted(all_chars))

❗ Rows that were not grouped:
   ΔG(cf) - ΔG(lin)  ΔG_abs_diff  ΔG_group
61      80.08-27.14        52.94       NaN
Number of unique characters: 9
List of unique characters: ['\n', 'A', 'C', 'G', 'T', 'a', 'c', 'g', 't']


### 위 코드에서 63번 행의 값은 범주에 속하지 않는 것을 확인했으며 그룹화되지 않아 제거

2. /n을 제거하고 소문자는 대문자로 변경하여 A, C, G, T만 남김

In [ ]:
# Sequences 열 처리: 결측행 제거, 대문자 변환, 줄 바꿈 문자 제거
sequences = df["Sequences"].dropna().astype(str)
cleaned_sequences = sequences.str.upper().str.replace('\n', '', regex=False)

# sequences 열에서 고유 문자 추출
all_chars = set("".join(cleaned_sequences))

# 고유 문자와 개수 출력
print("Number of unique characters:", len(all_chars))
print("List of unique characters:", sorted(all_chars))

Number of unique characters: 4
List of unique characters: ['A', 'C', 'G', 'T']


3. 시퀀스가 가장 긴 행을 찾고 길이와 내용 확인

In [ ]:
# 가장 긴 시퀀스 찾기
max_length = cleaned_sequences.str.len().max()

# 길이 출력
longest_rows = cleaned_sequences[cleaned_sequences.str.len() == max_length]

# 시퀀스가 가장 긴 행과 길이, 내용 출력
for idx, seq in longest_rows.items():
    print(f"Row index in DataFrame: {idx}")  # Add 2 to the index to get the actual row number in the CSV file (e.g., 205 + 2 = 207)
    print(f"Sequence length: {len(seq)}")
    print(f"Sequence contents: {seq[:100]}...")  # If it's too long, only the first 100 characters will be shown

Length of the longest sequence: 16584

📍 Row index in DataFrame: 205
🧬 Sequence length: 16584
🧬 Sequence contents: ATGGCTCAACTATTGGTAACTGATGTGCCAGTAGTTGATGGTATTATGGACATTGAAAATTATGCACCTGACATGGTGTATGTGGAACCTGAGAAGGAAA...


## One-hot encoding 적용

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 고유 문자 추출
all_chars = sorted(set("".join(cleaned_sequences)))

# OneHotEncoder 초기화
encoder = OneHotEncoder(categories=[all_chars], sparse_output=False, dtype=int, handle_unknown='ignore')
# 문자 리스트를 열 벡터로 변환
encoder.fit(np.array(all_chars).reshape(-1, 1))

# 최대 길이 시퀀스 계산
max_length = cleaned_sequences.str.len().max()

# 원핫 인코딩 및 패딩 실행
encoded_sequences = []

for seq in cleaned_sequences:
    chars = np.array(list(seq)).reshape(-1, 1)
    onehot = encoder.transform(chars)
    
    # 최대 길이 시퀀스의 길이에 맞춰 제로 패딩
    pad_len = max_length - onehot.shape[0]
    if pad_len > 0:
        padding = np.zeros((pad_len, len(all_chars)), dtype=int)
        onehot = np.vstack([onehot, padding])

    encoded_sequences.append(onehot)

# 인코딩된 시퀀스를 numpy 배열로 변환
encoded_array = np.stack(encoded_sequences)
print("One-hot encoding complete. shape =", encoded_array.shape)

✅ One-hot encoding complete. shape = (307, 16584, 4)


In [ ]:
# 첫 행 확인
sample_index = 0
sample = encoded_array[sample_index]  # shape: (최대 길이, 고유 문자 수)

print(f"Encoding shape of sample {sample_index}:", sample.shape)
print("Partial encoded array:")
print(sample[:10])  # 인코딩된 앞 10개만 출력

Encoding shape of sample 0: (16584, 4)
Partial encoded array:
[[0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]]


In [ ]:
# 문자로 되돌렸을 때 앞 10개 확인
reversed_chars = [all_chars[np.argmax(row)] for row in sample[:10]]
print("Recovered characters:", "".join(reversed_chars))

Recovered characters: GGCAGACAGA


In [ ]:
# (shape: [데이터 수, 최대 길이, 고유 문자 수])

# .npy format으로 저장
np.save("/content/drive/MyDrive/mini_project/Karl/data/encoded_sequences.npy", encoded_array) # 저장 경로
print("Save complete: encoded_sequences.npy")

✅ Save complete: encoded_sequences.npy


In [ ]:
# .npy 파일 불러오기
encoded_array = np.load("/content/drive/MyDrive/mini_project/Karl/data/encoded_sequences.npy")

# shape 출력
print("Full array shape:", encoded_array.shape)

Full array shape: (307, 16584, 4)
Number of sequences (rows): 307


## 모델 학습

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# === 1. .npy 파일 불러오기 ===
# .npy 파일 경로
npy_path = "/content/drive/MyDrive/mini_project/Karl/data/encoded_sequences.npy"

# 데이터 불러오기
X = np.load(npy_path)  # shape: (number of samples, max sequence length, number of characters)

# === 2. 3D에서 2D로 변환 ===
X_flat = X.reshape(X.shape[0], -1)  # shape: (데이터 수, 최대 길이)

print("X_flat shape:", X_flat.shape)

# === 3. 라벨 불러오기(ΔG_group) ===
# 'ΔG_group' 열 포함된 파일 불러오기
df_labels = pd.read_csv("/content/drive/MyDrive/mini_project/Karl/data/palindrome_with_group.csv")

# the ΔG_group 열에서 결측행 제거
df_clean = df_labels.dropna(subset=["ΔG_group"])

# 라벨만 추출하고 정수로 변환
y = df_clean["ΔG_group"].astype(int).values

# === 4. 훈련/테스트셋 분할 ===
X_train, X_test, y_train, y_test = train_test_split(
    X_flat, y, test_size=0.2, random_state=42
)

# === 5. Random Forest 분류기 학습 ===
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# === 6. 모델 평가 ===
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("✅ Random Forest Accuracy:", round(accuracy * 100, 2), "%")

X_flat shape: (307, 66336)
✅ Random Forest Accuracy: 67.74 %


In [ ]:
# 입력값 확인 = encoded Sequences
X_flat

array([[0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
# 출력값 확인 = ΔG_group
y

array([4, 4, 5, 4, 4, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 2, 4, 4, 3, 4,
       4, 4, 2, 3, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 5, 4, 3, 4, 4, 4,
       4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 3, 3, 3, 4, 3,
       3, 3, 3, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4,
       4, 4, 3, 4, 3, 3, 4, 4, 3, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 5, 4, 4,
       4, 4, 4, 4, 3, 3, 3, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 4, 3, 4, 3,
       3, 3, 4, 3, 3, 4, 4, 4, 3, 4, 4, 3, 4, 3, 4, 4, 3, 4, 5, 4, 4, 4,
       4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4,
       5, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 5, 4, 3, 3, 4, 4, 4, 3, 4, 4,
       4, 4, 3, 4, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 3, 5, 4, 4, 3, 3, 3,
       4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 3, 4, 4, 3, 4, 4, 4, 4, 3, 4,
       4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 3,
       2, 3, 3, 4, 3, 5, 4, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 4,
       4, 3, 3, 4, 5, 4, 4, 3, 4, 3, 4, 4, 4, 3, 4,